In [5]:
db = None

In [169]:
from collections import namedtuple
import sqlite3


def namedtuple_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    cls = namedtuple("Row", fields)
    return cls._make(row)


class MathEvalDB:
  def __init__(self, connection = None):
    if connection is None:
      connection = ':memory'
    if type(connection) == str:
      connection = sqlite3.connect(connection)
    self.connection = connection
    self.connection.row_factory = namedtuple_factory
    #self.connection.row_factory = sqlite3.Row
    self.cursor = self.connection.cursor()
    self.init_tables()

  def commit(self):
    self.connection.commit()
    
  def rollback(self):
    self.connection.rollback()
    
  def init_tables(self):
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS models (
        id INTEGER PRIMARY KEY,
        name TEXT
      )
    ''')
    
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS problem_sets (
        id INTEGER PRIMARY KEY,
        name TEXT,
        comment TEXT
      )
    ''')
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS problems (
        id INTEGER PRIMARY KEY,
        input TEXT,
        ideal TEXT,
        rubric TEXT,
        problem_set_id INTEGER,
        FOREIGN KEY(problem_set_id) REFERENCES problem_sets(id)
      )
    ''')
    
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS submission_sets (
        id INTEGER PRIMARY KEY,
        name TEXT,
        comment TEXT,
        model_id INTEGER,
        FOREIGN KEY(model_id) REFERENCES models(id)
      )
    ''')
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS submissions (
        id INTEGER PRIMARY KEY,
        completion_json TEXT,
        score NUMERIC,
        comment TEXT,
        problem_id INTEGER,
        submission_set_id INTEGER,
        FOREIGN KEY(problem_id) REFERENCES problems(id),
        FOREIGN KEY(submission_set_id) REFERENCES submission_sets(id)
      )
    ''')
    
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS evaluation_sets (
        id INTEGER PRIMARY KEY,
        name TEXT,
        comment TEXT,
        model_id INTEGER,
        FOREIGN KEY(model_id) REFERENCES models(id)
      )
    ''')
    self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS evaluations (
        id INTEGER PRIMARY KEY,
        completion_json TEXT,
        score NUMERIC,
        comment TEXT,
        submission_id INTEGER,
        evaluation_set_id INTEGER,
        FOREIGN KEY(submission_id) REFERENCES submissions(id),
        FOREIGN KEY(evaluation_set_id) REFERENCES evaluation_sets(id)
      )
    ''')


  def list_tables(self):
    return self.cursor.execute('''
      SELECT name
      FROM sqlite_schema
      WHERE type ='table' AND name NOT LIKE 'sqlite_%'
    ''').fetchall()

  def find_table(self, name):
    return self.cursor.execute('''
      SELECT name
      FROM sqlite_schema
      WHERE type ='table' AND name NOT LIKE 'sqlite_%' AND name = :name
    ''', {'name': name}).fetchall()
    
  def _and_str(self, **params):
    return ' AND '.join(f'{key} = :{key}' for key in params.keys())
  
  def list_table_entries(self, table_name, **params):
    table = self.find_table(table_name)
    if table:
      if params:
        and_str = self._and_str(**params)
        where_str = f'WHERE {and_str}'
        query_str = f'SELECT * FROM {table_name} {where_str}'
        return self.cursor.execute(query_str, params).fetchall()
      else:
        query_str = f'SELECT * FROM {table_name}'
        return self.cursor.execute(query_str).fetchall()
    else:
      raise KeyError(f'table "{table_name}" not found')

  
  def find_model(self, name):
    return self.cursor.execute('SELECT * FROM models WHERE name = :name', {'name': name}).fetchone()
    
  def add_model(self, name):
    model = self.cursor.execute('SELECT * FROM models WHERE name = :name', {'name': name}).fetchone()
    if not model:
      self.cursor.execute('INSERT INTO models (name) VALUES (:name)', {'name': name})

  def list_models(self, **params):
    return self.list_table_entries('models', **params)
    
    
  def find_problem_set(self, name):
    return self.cursor.execute('SELECT * FROM problem_sets WHERE name = :name', {'name': name}).fetchone()
    
  def add_problem_set(self, name):
    problem_set = self.cursor.execute('SELECT * FROM problem_sets WHERE name = :name', {'name': name}).fetchone()
    if not problem_set:
      self.cursor.execute('INSERT INTO problem_sets (name) VALUES (:name)', {'name': name})

  def list_problem_sets(self, **params):
    return self.list_table_entries('problem_sets', **params)

    
    
  def find_problem_set(self, name):
    return self.cursor.execute('SELECT * FROM problem_sets WHERE name = :name', {'name': name}).fetchone()
    
  def add_problem_set(self, name):
    problem_set = self.cursor.execute('SELECT * FROM problem_sets WHERE name = :name', {'name': name}).fetchone()
    if not problem_set:
      self.cursor.execute('INSERT INTO problem_sets (name) VALUES (:name)', {'name': name})

  def list_problem_sets(self, **params):
    return self.list_table_entries('problem_sets', **params)





In [170]:
if db is None:
  db = MathEvalDB('test.db')
else:
  db = MathEvalDB(db.connection)

In [171]:
db.list_tables()

[Row(name='models'),
 Row(name='problem_sets'),
 Row(name='problems'),
 Row(name='submission_sets'),
 Row(name='submissions'),
 Row(name='evaluation_sets'),
 Row(name='evaluations')]

In [172]:
db.find_table('models')

[Row(name='models')]

In [167]:
model = db.find_model('example')
model.name if model else None

'example'

In [168]:
model._fields

('id', 'name')

In [154]:
db.add_model('example')
db.commit()

In [155]:
db.list_models()

[Row(id=1, name='example')]

In [156]:
problem_set = db.find_problem_set('example')
problem_set.name if problem_set else None

'example'

In [157]:
db.add_problem_set('example')

In [158]:
problem_set = db.find_problem_set('example')
problem_set.name if problem_set else None

'example'

In [159]:
db.list_table_entries('problem_sets', name='example')

[Row(id=1, name='example', comment=None)]

In [12]:
list(db.cursor.execute('drop table problems'))

[]

In [109]:
_d = dict(fu = 'bar', foo = 'baz')
' AND '.join(f':{key}' for key in _d.keys())

':fu AND :foo'